In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

import evaluate
import merge
import load

from sklearn import preprocessing
from sklearn import svm
from sklearn import grid_search
from sklearn import cross_validation
from sklearn import pipeline
from sklearn import tree
from sklearn import metrics

In [17]:
# Load evaluation data
test_columns = ['returnQuantity', 'articleID', 'productGroup', 'customerID', 'voucherID', 'returnQuantityMultilabel']
test_predictions = merge.merged_predictions(test=True, keep_columns=test_columns)
test_train = evaluate.test_complement(test_predictions)

# Load classification data
class_columns = ['articleID', 'productGroup', 'customerID', 'voucherID', 'quantity']
class_predictions = merge.merged_predictions(keep_columns=class_columns)
class_train = load.orders_train()

In [3]:
multilabel = class_train[(class_train.returnQuantity > 0) & (class_train.quantity > 0)]
multilabel = merge.shuffle(multilabel)

Classification based on mean return quantities for a given quantity which turned out to perform better man other classifiers.

In [4]:
def score(cl, X, y):
    ytick = cl.predict(X)
    score = np.sum(np.abs(y - ytick))
    return 1./score

In [5]:
def classify(quantity):
    if quantity == 1:
        return 1
    elif quantity == 2:
        return 1
    elif quantity == 3:
        return 2
    elif quantity == 4:
        return 2
    elif quantity == 5:
        return 3
    else:
        print('wtf')
        raise ValueError

In [6]:
multilabel['baseline'] = multilabel.quantity.apply(lambda x: 1)
multilabel['predAvg'] = multilabel.quantity.apply(classify)
baseline = merge.dmc_cost(multilabel.baseline, multilabel.returnQuantity)
pred_avg = merge.dmc_cost(multilabel.predAvg, multilabel.returnQuantity)
baseline, pred_avg

(4350, 4172)

In [7]:
X = multilabel[['quantity', 'productGroup', 'price', 'rrp']].as_matrix()
y = np.squeeze(multilabel.returnQuantity.as_matrix())

In [8]:
clfT = tree.DecisionTreeClassifier()
tree_params = dict(criterion=['gini', 'entropy'], max_features=[2, 4], 
                   max_depth=[2, 4, 8, 100], min_samples_split=[2, 4, 8, 30, 100],
                   min_samples_leaf=[1, 4, 32, 100])
gsT = grid_search.RandomizedSearchCV(clfT, tree_params, n_iter=200)
gsT.fit(X[:500000], y[:500000])
gsT.score(X, y)

0.99669453916014972

In [9]:
merge.dmc_cost(y, gsT.predict(X))

4298

# Avg prediction works best because Decision Tree would only overfit
A regressor would be more suitable for this particular case and might outperform the avg prediction. On ther other hand side it seems thats only possible with overfitting especially with only few features as used here.

Final prediction works as follows

In [28]:
def classify(row):
    print(row)
    # if prediction is False/0
    if not row.prediction:
        return 0
    # else
    elif quantity == 1:
        return 1
    elif quantity == 2:
        return 1
    elif quantity == 3:
        return 2
    elif quantity == 4:
        return 2
    elif quantity == 5:
        return 3
    else:
        print('wtf')
        raise ValueError